# Coordinates with velocity information

- https://docs.sunpy.org/en/stable/topic_guide/coordinates/velocities.html

Velocity information can be added to any coordinate. When the coordinate is transformed to a different coordinate frame, the velocity vector will be transformed as appropriate. Be aware that the transformation framework does not take any velocity information into account when transforming the position vector.

## Creating a SkyCoord with velocity

Velocity information can be added as keyword arguments to SkyCoord. For sunpy’s frames, the names of the velocities components are the names of the position components prepended by “d_”, e.g.,:

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

import sunpy.coordinates

In [ ]:
sc = SkyCoord(lon=10*u.deg, lat=20*u.deg, distance=1*u.AU,
              d_lon=3*u.arcmin/u.week, d_lat=4*u.arcmin/u.d, d_distance=5*u.km/u.min,
              frame='heliocentricinertial', obstime='2021-01-01')

In [ ]:
sc

<SkyCoord (HeliocentricInertial: obstime=2021-01-01T00:00:00.000): (lon, lat, distance) in (deg, deg, AU)
    (10., 20., 1.)
 (d_lon, d_lat, d_distance) in (arcsec / s, arcsec / s, km / s)
    (0.00029762, 0.00277778, 0.08333333)>

## Querying velocity information

sunpy has functions to query the positions of planets or other objects (e.g., get_body_heliographic_stonyhurst()). For any of these functions, if include_velocity=True is specified, the returned coordinate will include velocity information, e.g.,:

In [ ]:
from sunpy.coordinates import get_body_heliographic_stonyhurst

In [ ]:
get_body_heliographic_stonyhurst('mercury', '2021-01-01')

<HeliographicStonyhurst Coordinate (obstime=2021-01-01T00:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
    (-156.46460438, -1.38836399, 0.43234904)>

In [ ]:
get_body_heliographic_stonyhurst('mercury', '2021-01-01', include_velocity=True)

<HeliographicStonyhurst Coordinate (obstime=2021-01-01T00:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
    (-156.46460438, -1.38836399, 0.43234904)
 (d_lon, d_lat, d_radius) in (arcsec / s, arcsec / s, km / s)
    (0.09138133, 0.00720229, -7.25136169)>

## Transforming velocities

The transformation of the velocity vector between two coordinate frames takes into account two effects:

- The change in the direction of the velocity vector due to a change in the orientation of the axes between the two frames.

- The “induced” velocity due to the time dependence of the frames themselves.

### Orientation change

To illustrate the orientation change, let’s start with the SkyCoord created at the beginning, which was defined in the HeliocentricInertial frame. We transform to Astropy’s HCRS frame, which is a different inertial frame that is also centered at the Sun: